In [2]:
# starter code already provided by Kaggle

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-science-interview-q-and-a-treasury/dataset.csv
/kaggle/input/gemma/keras/gemma_2b_en/2/config.json
/kaggle/input/gemma/keras/gemma_2b_en/2/tokenizer.json
/kaggle/input/gemma/keras/gemma_2b_en/2/metadata.json
/kaggle/input/gemma/keras/gemma_2b_en/2/model.weights.h5
/kaggle/input/gemma/keras/gemma_2b_en/2/assets/tokenizer/vocabulary.spm
/kaggle/input/gemma/pytorch/2b/1/config.json
/kaggle/input/gemma/pytorch/2b/1/gemma-2b.ckpt
/kaggle/input/gemma/pytorch/2b/1/tokenizer.model
/kaggle/input/data-assistants-with-gemma/submission_categories.txt
/kaggle/input/data-assistants-with-gemma/submission_instructions.txt
/kaggle/input/parquetfile-python-25k/0000.parquet


In [3]:
import pandas as pd
import re


# Install Keras 3 last. 
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

import keras
import keras_nlp

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.
tensorflowjs 4.16.0 requires packaging~=23.1, but you have packaging 21.3 which is incompatible.


2024-03-01 03:37:46.858858: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 03:37:46.858960: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 03:37:47.006391: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
#Creating the model using the from_preset method
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [5]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [6]:
data = pd.read_csv('/kaggle/input/data-science-interview-q-and-a-treasury/dataset.csv')

In [7]:
data.head()

,question,answer
0,What is supervised machine learning? 👶,Supervised learning is a type of machine learn...
1,What is regression? Which models can you use t...,Regression is a part of supervised ML. Regress...
2,What is linear regression? When do we use it? 👶,Linear regression is a model that assumes a li...
3,What are the main assumptions of linear regres...,There are several assumptions of linear regres...
4,What’s the normal distribution? Why do we care...,The normal distribution is a continuous probab...


In [8]:
#Preparing the dataset for fine-tuning
dataset = []
    
for index, row in data.iterrows():
    question, answer = row['question'], row['answer']
    template = (f"Question:\n{question}\n\nAnswer:\n{answer}")
    dataset.append(template)

In [9]:
#Enabling LoRA for the model and setting the LoRA rank to 64
# we could also do a smaller rank like 4 or 16 for computational efficiency
gemma_lm.backbone.enable_lora(rank=64)

In [10]:
# Limiting the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use of AdamW - a common optimizer for transformer models
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Excluding layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()])

gemma_lm.fit(dataset, epochs=15, batch_size=1)

Epoch 1/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 145s 736ms/step - loss: 0.4805 - sparse_categorical_accuracy: 0.5616
Epoch 2/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 122s 733ms/step - loss: 0.4116 - sparse_categorical_accuracy: 0.5932
Epoch 3/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 122s 734ms/step - loss: 0.3933 - sparse_categorical_accuracy: 0.6029
Epoch 4/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 122s 734ms/step - loss: 0.3698 - sparse_categorical_accuracy: 0.6185
Epoch 5/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 122s 734ms/step - loss: 0.3357 - sparse_categorical_accuracy: 0.6457
Epoch 6/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 122s 733ms/step - loss: 0.2926 - sparse_categorical_accuracy: 0.6853
Epoch 7/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 143s 860ms/step - loss: 0.2471 - sparse_categorical_accuracy: 0.7297
Epoch 8/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 122s 733ms/step - loss: 0.2091 - sparse_categorical_accuracy: 0.7702
Epoch 9/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 122s 733ms/step - loss: 0.1753 - sparse_categorical_accuracy: 0.8055
Epoch 10/15
166/166

In [11]:
gemma_lm.save('version_finetunded_ds.keras')

In [14]:
print(gemma_lm.generate("What is a Confusion Matrix", max_length=256))


What is a Confusion Matrix? What does it represent?

A confusion matrix is a table that summarizes the predictions of a model and the ground truth labels.

It consists of three columns: True Positives (TP), True Negative (TN), and False Positive (FP).

* TP is the number of true positives, which is when the model predicts a positive result and the ground truth is also positive.
* TN is the number of true negatives, which is when the model predicts a negative result and the ground truth is also negative.
* FP is the number of false positives, which is when the model predicts a positive result and the ground truth is also negative, or equivalently, when a negative result is predicted.
* Model accuracy = TN + TP / (TP + FP)
* Precision = TP / (TP + FP)
* Recall = TN / (TP + TN)

Model accuracy is a bit misleading because it doesn't tell you anything about the model's performance on negative examples. Precision and recall are ways of measuring how well a model is distinguishing between pos

In [29]:
print(gemma_lm.generate("What is the significance of p-value?", max_length=256))

What is the significance of p-value?

Answer:

The p-value is a measure of the significance of a given result. It tells you how likely it is that the result could be random or unimportant, and thus how significant it actually is.


In [33]:
print(gemma_lm.generate("what is bias in data science", max_length = 256))

what is bias in data science? how it affects our models?

Answer:

Bias in data science is an error introduced by the model itself. It affects the model's ability to generalize and make accurate predictions.


### Let's try this - now move on to finetuning the model to write Python code

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [34]:
file = pd.read_parquet("/kaggle/input/parquetfile-python-25k/0000.parquet")

In [35]:
data=file.apply(lambda row:f"Instruction:\n{row.instruction}\n\nResponse:\n{row.output}",axis=1).values.tolist()[:1000]

In [36]:
print(data[1])

Instruction:
Create a shopping list based on my inputs!

Response:
```python
shopping_list = {}
while True:
    item = input('Enter an item or type 'done' to finish: ')
    if item == 'done': break
    quantity = input(f'Enter the quantity for {item}: ')
    shopping_list[item] = quantity
print(f'Your shopping list: {shopping_list}')
```


In [37]:
gemma_lm.preprocessor.sequence_length = 128

optimizer = keras.optimizers.AdamW(
    learning_rate=5e-6,
    weight_decay=0.01,
)

optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 248s 227ms/step - loss: 1.0463 - sparse_categorical_accuracy: 0.6993


In [39]:
gemma_lm.save("version_finetuned_code.keras")


In [40]:
instruction="write a code for creating a list in python"
response=""
prompt = f"Instruction:\n{instruction}\n\nResponse:\n{response}"
print(gemma_lm.generate(prompt, max_length=128))

Instruction:
write a code for creating a list in python

Response:
```python
import numpy as np
lst = np.array([])
```
